Multidimensional data
=====================

The **Basics** tutorial has dealt with reading data, doing arithmetic,
intermediate inspection, and finally writing the results back to disk.

The example focused on familiar two dimensional data. However, xarray's power
becomes more apparent  when dealing with problems that have more than two
dimensions.

Once again, we've prepared some example data. This time around, it's a
geological layer model that one of your geological friends might have cooked
up, as well as some simulation results.

The ``imod`` package provides functions to easily open these data, even
if they are stored in separate two-dimensional files.

In [ ]:
import imod
# Shut up some annoying warnings as well
import warnings
warnings.simplefilter(action="ignore", category=RuntimeWarning)

In [ ]:
top = imod.idf.open("data/layers/top_l*.idf")
bot = imod.idf.open("data/layers/bot_l*.idf")

print(top)
print("\n")
print(bot)

We're dealing with three-dimensional data now:

1. layer
2. y
3. x
        
The ``imod.idf.open()`` function automatically parses the file names to infer
layer numbers and timestamps, if applicable. It does so by assuming the files have
been named optionally with datetime and layer number (prepended by an "L"),
separated by underscores, where time and layer are both optional.

Of course, there are many cases where file names do not agree with these naming
rules. ``imod.idf.open()`` provides some options in this case; [check the
documentation](https://imod.xyz/api/idf.html#imod.idf.open).

Anyhow, let's compute layer thickness, and check the thickness of the first
layer.

In [ ]:
thickness = top - bot
thickness.sel(layer=1).plot()

``.sel()`` selects values along a coordinate (in this case layer), based on value.
``.isel()`` selects values along a coordinate (in this case layer), based on index. 

Python's indexing is 0-based, so to select the first layer by index, we use:

In [ ]:
thickness.isel(layer=0).plot()

General indexing rules apply. To get the last one, use -1 as the index:

In [ ]:
thickness.isel(layer=-1).plot()

Now, unfortunately, our friendly geologist didn't get the memo that we'd like 14
layers, rather than 7. We'll have to improvise an additional 7 layers!

This is pretty easy: we simply select every layer twice, thereby creating a
fourteen layered model. Of course, we'll have to halve the layer thickness to
preserve total thickness.

In [ ]:
import numpy as np

doubled_layers = np.repeat(thickness["layer"], 2)
new_thickness = 0.5 * thickness.sel(layer=doubled_layers)

print(new_thickness)

The layer coordinates aren't quite right yet, so we'll update those:

In [ ]:
new_thickness["layer"] = np.arange(1, 15)

Now that have a fourteen layer thickness model, we can multiply the thicknesses
by hydraulic conductivity to compute transmissitivities.

Somebody has left them in ESRII ASCII format, but apart from slower read times,
it's not an issue.

In [ ]:
kh = imod.rasterio.open("data/kh/kh_l*.asc")
kD = new_thickness * kh

What does the total transmissitivity, summed over all layers look like?

Xarray provides methods to perform mathematical operations over one or more
dimensions.

In [ ]:
total_kD = kD.sum("layer")
print(total_kD)
total_kD.plot()

``.sum()`` is a reducing function: the output is `reduced` by one or more
dimensions.  Similarly, ``.mean()``, ``.min()``, and ``.max()``, ``.count()``
are also reducing functions.

To compute the mean transmissivity per layer, we reduce over ``y`` and ``x``:

In [ ]:
mean_kD = kD.mean(["y", "x"])
print(mean_kD.compute())

There are more functions that operate over one or more dimensions, although
they do not reduce.  These are for example ``.diff()`` or ``.cumsum()`` (for
difference and cumulative sum).

``.cumsum()`` can be used to here to compute new tops and bottoms.

In [ ]:
new_bot = top.sel(layer=1) - new_thickness.cumsum("layer")
new_top = new_bot + new_thickness

The ``imod`` package provides some utilities for saving multi-dimensional data
to geospatial formats. For IDFs, we have defined the ``imod.idf.save()``
function.

In [ ]:
imod.idf.save("output/thickness", new_thickness)

The ``.save()`` function automatically creates the necessary IDF files to store
all the layers. The file names are automatically generated by appending the
additional dimensions. It also adds the ``.idf`` extension.

The ``imod.idf.save()`` function differs from the ``imod.idf.write()`` function.
The ``write()`` function only writes a `single` IDF file, which must therefore
be two dimensional, with as dimensions only ``y`` and ``x``. ``write()`` also
does not attempt to come up with a file name, but writes it to exactly the file
you specify.

Let's check the result of ``.save()``:

In [ ]:
import os
print(os.listdir("output"))

## Xarray, Dask, and lazy evaluation

As a careful observer, you might've noticed we called ``.compute`` above when printing the results of the mean transmissivity per layer:

```
mean_kD = kD.mean(["y", "x"])
print(mean_kD.compute())
```

This is what happens when we don't call ``.compute()``:

In [ ]:
print(mean_kD)

We don't see our output here as an array of seven numbers as before:

``array([ 14.43255,  40.48827, 135.03757, 130.95311, 229.52626, 190.12553, 216.42268], dtype=float32)``
       
Instead we get:

``dask.array<mean_agg-aggregate, shape=(7,), dtype=float32, chunksize=(1,), chunktype=numpy.ndarray>``

The reason is [lazy evaluation](https://en.wikipedia.org/wiki/Lazy_evaluation). In lazy evaluation, operations (like subtractions and addition) are not executed immediately. Rather, the operation is added to a "to do list".
The ``imod.idf.open`` operation is lazy in terms of loading data; the data of the IDFs isn't read into memory immediately.

Let's check this "to do list" for ``top``:

In [ ]:
top.data.visualize()

The individual IDF files are read (one read statement for every IDF), and then they are stacked to form the ``top`` DataArray.

For ``thickness``, we expect to see:

1. Creation of the ``top`` array.
2. Creation of the ``bot`` array.
3. Subtraction of the two to form ``thickness``.

These steps are clearly visible in the following graph:

In [ ]:
thickness.data.visualize()

Dask arrays are like numpy arrays, except that result are not computed **eagerly** and a single array may consists of multiple **chunks**. This has two primary benefits: 

1. Calculations can be efficiently parallelized (e.g. one chunk per processor).
2. Larger than memory datasets can be handled, as long as the **chunks** fit within memory.

To give a specific example: we could open many more IDFs than fit within memory, and still compute mean values with a single operation!

In Jupyter notebooks, there's a fancy HTML representation of these lazy, chunked arrays:

In [ ]:
thickness.data

To force the computation, we call ``.compute()``. Essentially, ``.compute()`` turns a dask array into an ordinary numpy array. Generally, it isn't necessary to call ``.compute()`` yourself. For example, during plotting the result will be computed as well, or when writing to disk.

In [ ]:
print(type(thickness.data))
print(type(thickness.compute().data))

## Time

So far, we've only dealt with spatial dimensions: ``layer``, ``y``, and ``x``.
We're going to look at time varying data next. 

As mentioned above, the ``imod.idf.open()`` function automatically parses
timestamps, and we can open a simulation output as follows:

In [ ]:
head = imod.idf.open("data/head/head_*_l1.idf")
print(head)

The timestamps in the file names are automatically converted to a ``datetime``

If we select a single point in space and plot it, we automatically get a
timeseries:

In [ ]:
timeseries = head.sel(x=450_000, y=86_000, method="nearest")
timeseries.plot()

We can compare the simulated head with measurements.

Let's load some CSV data of a piezometer, of the location selected above,
and plot the measured and simulated head together.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("data/piezometer.csv", index_col=["date"], parse_dates=["date"])

fig, ax = plt.subplots()
timeseries.plot(ax=ax)
df["measured"].plot(ax=ax, style=".")

Next, have a look at how the results hold up spatially.

Using geopandas we can load a shapefile that contains the mean measurement value for a set of piezometers.
We can compute the mean simulated head by calling ``.mean("time")``.

We finish by plotting the measurements
on top of the mean simulation result.

In [ ]:
import geopandas as gpd
gdf = gpd.read_file("data/piezometers.shp")

mean_head = head.mean("time").squeeze("layer")

fig, ax = plt.subplots()
mean_head.plot.contourf(levels=np.arange(-5.0, 5.0), ax=ax)
gdf.plot(ax=ax, column="measured", cmap="viridis")

We can also easily make a scatter plot by selecting the values from ``mean_head`` based on x and y:

In [ ]:
gdf["simulated"] = imod.select.points_values(mean_head, x=gdf.geometry.x.values, y=gdf.geometry.y.values)
df = gdf[["simulated", "measured"]]
df.plot.scatter("measured", "simulated")
plt.plot([-6.0, -1.0], [-6.0, -1.0], linestyle='--', color="r")

Compute the absolute difference, and plot it spatially:

In [ ]:
gdf["abs_diff"] = (gdf["simulated"] - gdf["measured"]).abs()

fig, ax = plt.subplots()
mean_head.plot.contourf(levels=np.arange(-5.0, 5.0), ax=ax)
gdf.plot(ax=ax, column="abs_diff", cmap="viridis")

The ``imod`` package also supports writing tabular data to IPF files:

In [ ]:
gdf["x"] = gdf.geometry.x
gdf["y"] = gdf.geometry.y
to_ipf = gdf[["x", "y", "id", "measured", "abs_diff"]]
imod.ipf.save("output/piezometers.ipf", to_ipf)

In summary, we have:

* Loaded many IDFs into Python using ``imod`` functions.   
* Selected data by value and index.
* Used selection to create a new DataArray.   
* Applied mathematical operation over specific dimensions.
* Written data back to disk.
* Loaded time-dependent simulation data into Python.
* Compared it with (tabular) measurements.
* Written the resulting table to an IPF file.